<a href="https://colab.research.google.com/github/kapibara3chi/NLPTesForGC/blob/main/NLPTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# pip

In [ ]:
!pip  install -U ginza ja_ginza_electra

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.8 MB 11.8 MB/s 
     |████████████████████████████████| 2.2 MB 47.6 MB/s 
     |████████████████████████████████| 283 kB 37.3 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
     |████████████████████████████████| 1.1 MB 36.7 MB/s 
     |████████████████████████████████| 4.7 MB 39.6 MB/s 
     |████████████████████████████████| 7.6 MB 36.3 MB/s 
     |████████████████████████████████| 6.6 MB 39.5 MB/s 
     |████████████████████████████████| 182 kB 56.6 MB/s 
  Created wheel for SudachiDict-core: filename=SudachiDict_core-20221021-py3-none-any.whl size=71574782 sha256=13c5c7b142aee4fc47aa12b3caa6035fc05ae24f7069041f83979e450f23622c
  Stored in directory: /root/.cache/pip/wheels/ca/e3/ed/e78fecf6fd34349114d292242a16fc08d513fb32c2d9c5d786
  Created wheel for sudachitra: filename=SudachiTra-0.1.7-py3-none-any.whl size=266095 sha256=5e91

# main

In [ ]:
import ginza
from spacy import displacy
import spacy
class DependencyAnalysis:
    def __init__(self):
        self.nlp = spacy.load('ja_ginza_electra')

    def run(self, text):
        """係り受け解析"""
        doc = self.nlp(text)

        token_head_list = []

        for sent in doc.sents:
            for token in sent:
                token_head_list.append(
                    {"i": token.i, "orth": token.orth_, "base": token.lemma_, "head": token.head.i, "dep": token.dep_})

        displacy.render(doc, style='dep', jupyter=True, options={'distance': 90})

        return token_head_list
if __name__ == '__main__':
  dependency = DependencyAnalysis()
  dependency.run("下流あごをだすと分離は改善するが、コンタミが悪化する")